In [2]:
#패키지 설정                               
from sklearn.preprocessing import StandardScaler         #정규화 모듈 설정
from sklearn.model_selection import train_test_split       #학습용과 테스트용 분리
from sklearn.metrics import mean_absolute_error
from keras.models import Sequential        #딥러닝 모형화
from keras.layers import Dense           #층간 노드 모두 연결
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
df = pd.read_excel('/content/drive/MyDrive/대학원/1학기/1. 고급통계분석(김영순)/z. 기말고사 과제/주가 자료 원본/삼성전자 주가 관련 자료(엑셀)-3.xlsx')

print(df.head(8))

          일자     시가     고가     저가     종가  전일비  7 거래일 이내 최고 상승률    등락  주가등락률  \
0 2022-12-05  60900  61100  60000  60300    0                0  -100  -0.17   
1 2022-12-02  62500  62500  60400  60400    0                0 -2200  -3.51   
2 2022-12-01  63100  63200  62300  62600    1                0   400   0.64   
3 2022-11-30  60400  62200  60200  62200    1                0  1600   2.64   
4 2022-11-29  59900  60600  59800  60600    1                0   500   0.83   
5 2022-11-28  60500  60500  59800  60100    0                0  -900  -1.48   
6 2022-11-25  61400  61700  60800  61000    0                0  -400  -0.65   
7 2022-11-24  61200  61700  60900  61400    1                1   400   0.66   

        거래량  ...  사모펀드(KOSPI)  기타법인(KOSPI)    기준환율  환율 증감  Open(NASDAQ)  \
0  13767787  ...         -913           59  1292.6   -7.3   11380.99023   
1  15331184  ...         -776           31  1299.9    0.2   11308.37988   
2  16631445  ...          -37          191  1299.7  -19.1   114

In [9]:
df['7 거래일 이내 최고 상승률'].value_counts()

1    789
2    146
3     25
0      7
4      2
Name: 7 거래일 이내 최고 상승률, dtype: int64

In [10]:
df.set_index('일자',inplace=True)
df = df.astype('float')   #inplace 가 없음

In [11]:
y=df[['7 거래일 이내 최고 상승률']]   #목표변수의 데이터 프레임
y = y.astype(int)
y = y.reset_index()
y=y[['7 거래일 이내 최고 상승률']] 
print(y)

     7 거래일 이내 최고 상승률
0                  0
1                  0
2                  0
3                  0
4                  0
..               ...
964                2
965                3
966                2
967                2
968                2

[969 rows x 1 columns]


In [12]:
X = df.drop('7 거래일 이내 최고 상승률', axis=1)

In [13]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3, random_state=1)    #학습용과 데이터 분리(7:3)

In [14]:
#정규화 스케일러 생성
scalerX = StandardScaler()
#정규화 스케일러를 학습용 데이터에 맞춤
scalerX.fit(X_train)
#정규화 스케일러로 학습용 데이터를 변환
X_train_std=scalerX.transform(X_train)
#정규화 스케일러로 테스트용 데이터를 변화
X_test_std=scalerX.transform(X_test)

In [15]:
import numpy as np
y_train.reset_index(inplace=True)
y_train2 = np.array(y_train['7 거래일 이내 최고 상승률'])
y_test.reset_index(inplace=True)
y_test2 = np.array(y_test['7 거래일 이내 최고 상승률'])

In [16]:
import numpy as np
y_train2 = pd.get_dummies(y_train2)
y_test2 = pd.get_dummies(y_test2)

In [17]:
from sklearn import svm
rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(X_train_std, y_train['7 거래일 이내 최고 상승률'])

In [18]:
rbf_pred = rbf.predict(X_test_std)

In [19]:
y_test = y_test['7 거래일 이내 최고 상승률']
import sklearn
sklearn.metrics.confusion_matrix(y_test, rbf_pred)

array([[  0,   3,   0,   0,   0],
       [  0, 235,   0,   0,   0],
       [  0,  44,   0,   0,   0],
       [  0,   8,   0,   0,   0],
       [  0,   1,   0,   0,   0]])

In [20]:
from sklearn.metrics import *
acc = accuracy_score(y_test, rbf_pred)
print('accuracy_score : ',acc)

pre = precision_score(y_test, rbf_pred, average='weighted')
print('precision_score : ', pre)

recall = recall_score(y_test, rbf_pred, average='weighted')
print('recall_score: ', recall)

accuracy_score :  0.8075601374570447
precision_score :  0.6521533756096408
recall_score:  0.8075601374570447


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
